In [1]:
import pandas as pd


In [5]:
data = pd.read_pickle('/content/drive/MyDrive/artifact/transformed_data.pkl')

In [6]:
data.head()

,Outliers,PricingStrategy,ProductCategory_airtime,ProductCategory_financial_services,Value,Amount,ChannelId,SubscriptionId,AccountId,ProviderId,FraudResult
0,1,2,1.0,0.0,1000,1000.0,3,887,3957,6,0
1,1,2,0.0,1.0,20,-20.0,2,3829,4841,4,0
2,1,2,1.0,0.0,500,500.0,3,222,4229,6,0
3,-1,2,0.0,0.0,21800,20000.0,3,2185,648,1,0
4,1,2,0.0,1.0,644,-644.0,2,3829,4841,4,0


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190938 entries, 0 to 190937
Data columns (total 11 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   Outliers                            190938 non-null  int64  
 1   PricingStrategy                     190938 non-null  int64  
 2   ProductCategory_airtime             190938 non-null  float64
 3   ProductCategory_financial_services  190938 non-null  float64
 4   Value                               190938 non-null  int64  
 5   Amount                              190938 non-null  float64
 6   ChannelId                           190938 non-null  int64  
 7   SubscriptionId                      190938 non-null  int64  
 8   AccountId                           190938 non-null  int64  
 9   ProviderId                          190938 non-null  int64  
 10  FraudResult                         190938 non-null  int64  
dtypes: float64(3), int64(8)
me

In [12]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV

In [14]:


xgbclfr = XGBClassifier(random_state=42)


param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [1, 2, 3, 4, 5],
    'min_child_weight': [1, 2, 3, 4, 5],
    'gamma': [0.1, 0.2, 0.3, 0.4, 0.5],
    'learning_rate': [0.01, 0.1, 1, 1.5]
}

x,y=data.drop('FraudResult',axis=1),data['FraudResult']
randomizedsearch = RandomizedSearchCV(estimator=xgbclfr,param_distributions=param_grid,scoring='f1')
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)
randomizedsearch.fit(x_train,y_train)
print(randomizedsearch.best_estimator_)
print(randomizedsearch.best_params_)

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.4, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=5, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)
{'n_estimators': 100, 'min_child_weight': 5, 'max_depth': 4, 'learning_rate': 0.1, 'gamma': 0.4}


In [16]:
import pickle
with open('model.pkl', 'wb') as files:
    pickle.dump(randomizedsearch.best_estimator_, files)